In [1]:
from evaluation import load_model

In [2]:
import torch

selected_model = "32010753b6f3476eb51f55e6898a44e7"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, tokenizer, prompt_pattern = load_model(selected_model, device=device)

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
from src.model_training.datasets.experiments_sanitize.complete_sanitization import DefinitionTestSet, DefaultTrainValSet
DefinitionTestSet.prompt_pattern = prompt_pattern
dataset_test = DefinitionTestSet.create_dataset(tokenizer, shuffle=True, seed=42, subset_test=-1)

In [22]:
from transformers import pipeline

pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=device)

In [23]:
def data_generator():
    for item in dataset_test:
        yield item["prompt"]
        
strategies = [
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 1.1},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 1.2},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 1.3},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 1.4},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 1.5},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 1.8},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "repetition_penalty": 2.0, "temperature": 1.2},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 0.5},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 1.1},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 1.2},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 1.3},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 1.4},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 1.5},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 1.8},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 2.0, "temperature": 1.2},
        {"name": "Encoder Repetition penalty", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "encoder_repetition_penalty": 0.5},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 0.1},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 0.3},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 0.5},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 0.7},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 0.9},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 1.1},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 1.2},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 1.4},
        {"name": "Temperature", "do_sample": True, "num_beams": 5, "max_new_tokens": 50, "temperature": 1.5},
    {"name": "No repeat NGRAM", "no_repeat_ngram_size": 2}

]

for i, strategy in enumerate(strategies):
    test_predictions = {
        'title': dataset_test['title'],
        'context_sentence': dataset_test['context_sentence'],
        'context_word': dataset_test['context_word'],
        'gt': dataset_test['gt'],
        'prediction': []
    }
    
    used_strategy = dict(strategy)
    used_strategy.pop("name")
    for out in pipe(data_generator(), batch_size=32, **used_strategy):
        assert len(out) == 1
        test_predictions['prediction'].append(out[0]["generated_text"])
        print(f"{i}: {out[0]['generated_text']}")
        break

0: Sport: Ernüchterung eines Spielers, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft
1: Sport, Fußball: Zustand, in dem etwas (beispielsweise ein Spiel, ein Spiel, ein Spiel, ein Spiel, einen Wettkampf, einen Wettkampf, einen Wettkampf, einen Wettkampf, einen Wett
2: Sport: Ernüchterung von etwas
3: Sport: Ernüchterung eines Spielers, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft, einer Mannschaft
4: Sport, Fußball: Ernüchterung
5: bildungssprachlich: Ernüchterung
6: Sport: etwas, das in der Folge eines Wettkampfes zustande kommt
7: Sport, Fußball: das Erzeugen eines Spielers, eines Spielers, eines Spielers, eines Spielers, eines Spielers, eines Spielers, eines Spielers, eines Spielers oder eines Spieler
8: Sport, Fussball: Ernüchterung eines

KeyboardInterrupt: 

In [27]:
dataset_test['prompt'][0]

'Nach den letzten großartigen Erfolgen des FC Bayern erfolgte bei der Heimniederlage die Ernüchterung. Was ist die Definition von Ernüchterung.? '